In [33]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [34]:
airline_data = pd.read_csv("flight_data.csv", low_memory = False)

In [35]:
df = airline_data.loc[~airline_data["DepDelay"].isnull(), ["DepDelay", "Reporting_Airline", "Origin"]]


# Full Service Carriers vs Low Cost Carriers
 - https://www.researchgate.net/publication/287458692_Comparison_of_FSC_and_LCC_and_their_market_share_in_aviation

 - https://www.icao.int/sustainability/Documents/LCC-List.pdf

In [36]:
# https://www.icao.int/sustainability/Documents/LCC-List.pdf
f = open("list_of_LCCs.txt", "r")
LCC_data = f.read()
f.close()

In [37]:
LCC_data

"Global Air Transport Outlook to 2030 http://www.icao.int/sustainability/Pages/GATO2030.aspx\nICAO DATA+ http://stats.icao.int\nas of 13 June 2017\nemail: ecd@icao.int\nRegion Country of AOC Airline name\nICAO\ncode\nIATA\ncode\nBeginning of\noperation\nCeased\noperation\nFormer names Notes\nAfrica South Africa 1time Airline RNX T6 2004 2012\nAfrica Egypt Air Arabia Egypt RBG E5 2010 Operates flights on behalf of parent airlines Air Arabia and offers connections throught Sharjah to all Air Arabia destinations\nAfrica Morroco Air Arabia Maroc MAC 3O 2009\nAfrica Egypt Air Leisure ALD AL 2015\nAfrica Kenya Air Peace APK P4 2016\nAfrica Morroco Atlas Blue BMM 8A 2004 2009 Integrated in Royal Air Maroc\nAfrica Egypt Aviator Aviation AVV T9 2016\nAfrica Tanzania Fastjet FTZ FN 2012 Easyjet owns 5,5 %\nAfrica Kenya Five Forty Aviation FFV 5H 2009\nAfrica Kenya Jambojet JX 2014 2017\nAfrica Morroco Jet4you JFU 8J 2006 2012 Merged with JetairFly (Belgium)\nAfrica South Africa kulula.com CAW MN

In [38]:
import re

In [39]:
ICAO = re.findall(r" ([A-Z\d][A-Z\d][A-Z\d]) ", LCC_data)
LATA = re.findall(r" ([A-Z\d][A-Z\d]) ", LCC_data)

In [40]:
ICAO[:5], LATA[:5] #eh good enough
print(LATA)

['13', 'T6', 'E5', '3O', 'AL', 'P4', '8A', 'T9', 'FN', '5H', 'JX', '8J', 'MN', 'JE', 'N6', 'FA', 'C9', 'AQ', 'KI', 'E4', 'AK', 'I5', 'JW', 'D7', 'Z2', 'PA', 'BX', 'HD', 'IX', 'ZM', 'OP', 'RS', 'DG', '5J', 'EU', 'PN', 'QG', 'YM', 'ZE', 'SJ', 'G8', 'Y5', 'UO', 'VQ', '6E', 'QZ', '7C', 'S2', 'JQ', '3K', 'GK', 'BL', 'LJ', 'IT', 'KC', 'JT', '8L', 'OD', 'MJ', 'DD', 'XW', 'O8', 'OX', 'DJ', '2P', 'MM', 'PQ', 'DR', 'TZ', 'XO', 'BC', 'LQ', 'SG', '9C', 'IJ', '7G', 'NZ', 'FD', 'XJ', 'SL', 'VZ', 'TT', 'RI', 'TR', 'IT', '2T', 'TW', 'VA', 'VF', 'JW', 'VJ', 'VA', 'DJ', 'ZG', 'IW', 'AB', 'RE', 'SH', '4P', 'WO', '2B', 'KK', 'LZ', '0B', 'BV', 'KF', 'WW', 'C0', 'XG', 'CO', 'DE', '7H', 'D5', 'DI', 'BA', '2G', 'BA', 'DI', '5D', 'U2', 'DS', 'VE', 'EW', 'E2', 'F7', 'BE', 'BA', 'Y2', 'SH', 'LF', 'ST', '4U', 'GO', '2L', '5K', 'HC', '3L', '9X', 'LS', 'GX', 'NM', 'IG', 'ZB', '8I', 'HG', 'DY', '8Q', 'H9', 'DP', 'FR', 'NE', 'AG', '5P', 'XW', 'QS', 'NB', 'XQ', 'XG', 'TO', 'HV', 'TB', 'X3', 'TV', 'SN', 'SN', 'TV', 'VA

In [41]:
df["Is_LCC"] = df["Reporting_Airline"].isin(LATA)
df

,DepDelay,Reporting_Airline,Origin,Is_LCC
0,19.0,NW,MSP,False
1,-2.0,FL,MKE,True
2,14.0,MQ,GJT,False
3,0.0,DL,LAX,False
4,51.0,US,EWR,True
...,...,...,...,...
1999995,4.0,WN,LAS,True
1999996,0.0,CO,EWR,True
1999997,-6.0,US,CHS,True
1999998,8.0,WN,MDW,True


In [42]:
df["Is_LCC"].value_counts()

Is_LCC
False    1233786
True      730146
Name: count, dtype: int64

In [48]:
xy_object = sm.stats.CompareMeans(
    sm.stats.DescrStatsW(df.loc[df["Is_LCC"], "DepDelay"]), 
    sm.stats.DescrStatsW(df.loc[~df["Is_LCC"], "DepDelay"])
)

sm_result = xy_object.tconfint_diff(alpha=0.05, usevar='unequal'), 
sm_result

((0.45092253724394016, 0.6317632696991496),)

In [49]:
import scipy.stats as stats
stats.ttest_ind(
    df.loc[df["Is_LCC"], "DepDelay"], 
    df.loc[~df["Is_LCC"], "DepDelay"], 
    equal_var=False,
    alternative='greater'
),

(Ttest_indResult(statistic=11.734229482357215, pvalue=4.266582078145926e-32),)

In [50]:
df.loc[df["Is_LCC"], "DepDelay"].mean() - df.loc[~df["Is_LCC"], "DepDelay"].mean()

0.5413429034715449

In [51]:
df.loc[df["Is_LCC"], "DepDelay"].mean(), df.loc[~df["Is_LCC"], "DepDelay"].mean() 

(8.927488474907758, 8.386145571436213)

In [52]:
def run_permutation(df, category="category", operation=lambda x, y: 0, n=1000):
    random_df = df.copy(deep=True)
    sample_stat = operation(df, category)
    sim_stats = []
    for _ in range(1000):
        random_df["simulated"] = random_df[category].sample(frac=1).reset_index(drop=True)
        random_df
        sim_stats.append(
            operation(random_df.drop(category, axis=1), "simulated")
        )
    return np.array(sim_stats), sample_stat

diff_means = lambda df, category: df.groupby(category).mean().diff().iloc[1, 0]
sim_stats, sample_stat = run_permutation(df[["Is_LCC", "DepDelay"]], category="Is_LCC", operation=diff_means, n=1000)

In [53]:
np.sum(sim_stats > sample_stat) / len(sim_stats)

0.0

# Now just looking at pure delays

In [54]:
delay_df = df[df["DepDelay"] > 0]
xy_object = sm.stats.CompareMeans(
    sm.stats.DescrStatsW(delay_df.loc[df["Is_LCC"], "DepDelay"]), 
    sm.stats.DescrStatsW(delay_df.loc[~df["Is_LCC"], "DepDelay"])
)

sm_result = xy_object.tconfint_diff(alpha=0.05, usevar='unequal'), 
sm_result

((-3.035004686371858, -2.6333865837763692),)

In [55]:
import scipy.stats as stats
stats.ttest_ind(
    delay_df.loc[delay_df["Is_LCC"], "DepDelay"], 
    delay_df.loc[~delay_df["Is_LCC"], "DepDelay"], 
    equal_var=False,
    alternative='greater'
),

(Ttest_indResult(statistic=-27.662748598039453, pvalue=1.0),)

In [57]:
stats.ttest_ind(
    delay_df.loc[delay_df["Is_LCC"], "DepDelay"], 
    delay_df.loc[~delay_df["Is_LCC"], "DepDelay"], 
    equal_var=False,
    alternative='less'
),

(Ttest_indResult(statistic=-27.662748598039453, pvalue=1.1912487348341412e-168),)

In [60]:
diff_means = lambda df, category: df.groupby(category).mean().diff().iloc[1, 0]
sim_stats, sample_stat = run_permutation(delay_df[["Is_LCC", "DepDelay"]], category="Is_LCC", operation=diff_means, n=1000)
np.sum(sim_stats < sample_stat) / len(sim_stats)

0.0

# Question Do Non LCC carriers just have greater std in delays?

In [64]:
def run_permutation(df, category="category", operation=lambda x, y: 0, n=1000):
    random_df = df.copy(deep=True)
    sample_stat = operation(df, category)
    sim_stats = []
    for _ in range(1000):
        random_df["simulated"] = random_df[category].sample(frac=1).reset_index(drop=True)
        random_df
        sim_stats.append(
            operation(random_df.drop(category, axis=1), "simulated")
        )
    return np.array(sim_stats), sample_stat

diff_means = lambda df, category: df.groupby(category).std(ddof=0).diff().iloc[1, 0]
sim_stats, sample_stat = run_permutation(df[["Is_LCC", "DepDelay"]], category="Is_LCC", operation=diff_means, n=1000)

In [71]:
np.sum(sim_stats < sample_stat) / len(sim_stats), df.groupby("Is_LCC")[["DepDelay"]].std(ddof=0)

(0.0,
          DepDelay
 Is_LCC           
 False   34.755958
 True    28.967330)

In [68]:
sample_stat, sim_stats

(-5.788628252186832,
 array([-7.52469966e-01, -3.71919899e-01,  1.29996926e-01,  1.01706226e+00,
         1.89958012e-01, -1.18817472e+00, -1.41827360e-01, -4.16588229e-01,
         9.51971972e-02,  2.75382075e-02, -2.22891206e-01, -8.69683294e-01,
        -1.25478393e-01, -7.30861686e-03, -4.35899592e-01,  4.74044995e-01,
         3.72404455e-01, -9.95146694e-02,  6.58322613e-01, -1.08154095e-02,
         3.13234422e-01, -2.97436385e-01,  9.98363335e-02, -2.92662849e-02,
         1.24571165e+00,  1.03384721e+00, -2.53823570e-01, -1.17469046e-01,
         6.78732199e-02, -3.72483089e-02,  3.17377520e-01,  5.06625126e-01,
        -4.38573138e-01,  3.06980087e-01,  4.38120467e-01,  3.38677648e-01,
        -1.90683462e-01, -8.28685434e-01, -1.80840571e-02, -7.09433463e-01,
         7.83511591e-01, -2.16780115e-01,  3.03514438e-01,  8.78394290e-01,
         3.55097927e-01,  3.02610496e-02, -3.69322066e-01,  5.48315477e-02,
        -3.98769003e-01, -2.95888519e-01,  3.13919915e-01, -3.09458

# So an LCC is more consistent in when the airline actually leaves than other carrier types... Question, is the non_lcc category simliar then?

In [77]:
non_lcc_df = df.loc[~df["Is_LCC"]]
non_lcc_df.groupby("Reporting_Airline")["DepDelay"].mean()

Reporting_Airline
9E         7.753923
AA         8.442962
AS         5.854416
DL         7.559099
EA         8.392810
EV        12.664960
HA         0.135497
HP         8.315420
KH         1.592233
ML (1)     6.828457
MQ         8.979800
NW         6.063685
OH        10.727891
OO         8.134029
PA (1)     5.779018
PI        10.297435
UA        10.099443
XE         8.591354
Name: DepDelay, dtype: float64

In [78]:
df.loc[df["Is_LCC"]].groupby("Reporting_Airline")["DepDelay"].mean()

Reporting_Airline
B6    12.999784
CO     7.727288
DH     9.763825
F9    10.251869
FL     8.665500
G4    12.006192
NK    10.716395
PS     9.660070
TW     7.826627
TZ     5.698925
US     6.922162
VX    10.087857
WN     9.836225
YV    10.441027
YX     7.707586
Name: DepDelay, dtype: float64

In [79]:
df

,DepDelay,Reporting_Airline,Origin,Is_LCC
0,19.0,NW,MSP,False
1,-2.0,FL,MKE,True
2,14.0,MQ,GJT,False
3,0.0,DL,LAX,False
4,51.0,US,EWR,True
...,...,...,...,...
1999995,4.0,WN,LAS,True
1999996,0.0,CO,EWR,True
1999997,-6.0,US,CHS,True
1999998,8.0,WN,MDW,True


# Maybe lets manually determine what each type of carrier typically is

In [81]:
# https://gist.github.com/AndreiCalazans/390e82a1c3edff852137cb3da813eceb

airline_code_dictory = {
  "U2": "easyjet",
  "1T": "Bulgarian Air Charter",
  "Q5": "40-Mile Air",
  "4O": "Interjet",
  "7A": "Express Air Cargo",
  "JY": "Air Turks and Caicos",
  "JU": "Air Serbia",
  "QH": "Kyrgyzstan",
  "A8": "Benin Golf Air",
  "RV": "Caspian Airlines",
  "1B": "Abacus International",
  "W9": "Eastwind Airlines",
  "6U": "Air Ukraine",
  "E4": "Aero Asia International",
  "ZI": "Aigle Azur",
  "AE": "Mandarin Airlines",
  "OZ": "Ozark Air Lines",
  "8U": "Afriqiyah Airways",
  "Q9": "Afrinat International Airlines",
  "KI": "Adam Air",
  "QB": "Georgian National Airlines",
  "LD": "Air Hong Kong",
  "UX": "Air Europa",
  "NX": "Air Macau",
  "ZV": "Air Midwest",
  "HM": "Air Seychelles",
  "AF": "Air France",
  "SB": "Air Caledonie International",
  "EH": "SAETA",
  "ZW": "Air Wisconsin",
  "GN": "Air Gabon",
  "NQ": "Air Japan",
  "VD": "SwedJet Airways",
  "TT": "Tigerair Australia",
  "4N": "Air North Charter - Canada",
  "NZ": "Eagle Airways",
  "QM": "Air Malawi",
  "ML": "Midway Airlines (1976–1991)",
  "P8": "Pantanal Linhas Aéreas",
  "BM": "BMI Regional",
  "ZX": "Air Georgian",
  "G8": "Gujarat Airways",
  "7T": "Tobruk Air",
  "6V": "Mars RK",
  "NH": "All Nippon Airways",
  "TZ": "ATA Airlines",
  "2Q": "Air Cargo Carriers",
  "V7": "Volotea",
  "AB": "Air Berlin",
  "4D": "Air Sinai",
  "QN": "Air Armenia",
  "AI": "Air India Limited",
  "PJ": "Air Saint Pierre",
  "SZ": "Air Southwest",
  "8C": "Shanxi Airlines",
  "NF": "Air Vanuatu",
  "ZB": "Monarch Airlines",
  "CC": "Macair Airlines",
  "RB": "Syrian Arab Airlines",
  "TN": "Air Tahiti Nui",
  "SW": "Air Namibia",
  "AW": "Dirgantara Air Service",
  "PE": "People's Viennaline",
  "JM": "Jetstar Hong Kong Airways",
  "6G": "Air Wales",
  "TX": "Transportes Aéreos Nacionales",
  "IX": "Air India Express",
  "BT": "Air Baltic",
  "EL": "Ellinair",
  "YW": "Air Nostrum",
  "PX": "Air Niugini",
  "G9": "Air Arabia",
  "AC": "Air Canada",
  "AP": "Air One",
  "XT": "SkyStar Airways",
  "UM": "Air Zimbabwe",
  "S2": "Air Sahara",
  "TC": "Air Tanzania",
  "2J": "Air Burkina",
  "KM": "Air Malta",
  "YT": "Air Togo",
  "G4": "Allegiant Air",
  "M3": "North Flying",
  "O4": "Antrak Air",
  "GB": "ABX Air",
  "8V": "Wright Air Service",
  "8T": "Air Tindi",
  "JP": "Adria Airways",
  "A3": "Aegean Airlines",
  "2K": "Aerogal",
  "KD": "KD Avia",
  "KO": "Alaska Central Express",
  "VX": "Virgin America",
  "KH": "Aloha Air Cargo",
  "AA": "American Airlines",
  "AX": "Trans States Airlines",
  "AN": "Ansett Australia",
  "5W": "Astraeus",
  "VV": "Aerosvit Airlines",
  "WK": "Edelweiss Air",
  "QQ": "Reno Air",
  "FG": "Ariana Afghan Airlines",
  "Y2": "Flyglobespan",
  "SU": "Aeroflot Russian Airlines",
  "5Z": "VivaColombia",
  "5D": "DonbassAero",
  "1A": "Amadeus IT Group",
  "JJ": "LATAM Brasil",
  "PL": "Airstars",
  "8A": "Atlas Blue",
  "GD": "Air Alpha Greenland",
  "HT": "Aeromist-Kharkiv",
  "J2": "Azerbaijan Airlines",
  "U3": "Avies",
  "4Y": "Yute Air Alaska",
  "5A": "Alpine Air Express",
  "W4": "Aero Services Executive",
  "IZ": "Arkia Israel Airlines",
  "M6": "Amerijet International",
  "4A": "Air Kiribati",
  "EV": "ExpressJet",
  "HP": "Phoenix Airways",
  "VH": "Aeropostal Alas de Venezuela",
  "AM": "Aeroméxico",
  "TL": "Trans Mediterranean Airlines",
  "OY": "Omni Air International",
  "IW": "Wings Air",
  "J6": "AVCOM",
  "2D": "Aero VIP",
  "VB": "VIVA Aerobus",
  "OE": "Asia Overnight Express",
  "GV": "Aero Flight",
  "JW": "Vanilla Air",
  "2B": "Aerocondor",
  "4C": "Aires, Aerovías de Integración Regional, S.A.",
  "AR": "Aerolíneas Argentinas",
  "AS": "Alaska Airlines, Inc.",
  "OB": "Oasis International Airlines",
  "HC": "Iceland Express",
  "FO": "Airlines of Tasmania",
  "OS": "Austrian Airlines",
  "IQ": "Augsburg Airways",
  "RU": "SkyKing Turks and Caicos Airways",
  "MO": "Calm Air",
  "GR": "Gemini Air Cargo",
  "NO": "Neos",
  "AU": "Austral Líneas Aéreas",
  "AO": "Australian Airlines",
  "AV": "Avianca - Aerovías del Continente Americano S.A.",
  "A0": "Avianca Argentina",
  "O6": "Avianca Brazil",
  "K8": "Airlink Zambia",
  "B9": "Air Bangladesh",
  "HJ": "Hellas Jet",
  "AK": "Air Bridge Carriers",
  "D7": "FlyAsianXpress",
  "DJ": "Polynesian Blue",
  "I5": "AirAsia India",
  "EX": "Air Santo Domingo",
  "3G": "Atlant-Soyuz Airlines",
  "AZ": "Alitalia",
  "ZE": "Líneas Aéreas Azteca",
  "A2": "Cielos Airlines",
  "R7": "Aserca Airlines",
  "RX": "Aviaexpress",
  "MQ": "American Eagle Airlines",
  "ZS": "Sama Airlines",
  "FF": "Airshop",
  "VU": "Air Ivoire",
  "BP": "Air Botswana",
  "GS": "Tianjin Airlines",
  "VT": "Air Tahiti",
  "3N": "Air Urga",
  "VL": "Air VIA",
  "FK": "Keewatin Air",
  "G2": "Avirex",
  "V8": "Iliamna Air Taxi",
  "K6": "Khalifa Airways",
  "VE": "C.A.I. Second",
  "V5": "Royal Aruban Airlines",
  "CA": "Air China",
  "Q6": "Aero Condor Peru",
  "5F": "Fly One",
  "QC": "Air Corridor",
  "NV": "Air Central",
  "CV": "Cargolux",
  "CW": "Air Marshall Islands",
  "ZA": "Interavia Airlines",
  "AH": "Air Algérie",
  "ER": "Astar Air Cargo",
  "HO": "Juneyao Airlines",
  "EN": "Air Dolomiti",
  "NM": "Mount Cook Airlines",
  "EE": "Aero Airlines",
  "4F": "Air City",
  "EI": "Aer Lingus",
  "E8": "Alpi Eagles",
  "KY": "Air São Tomé and Príncipe",
  "PC": "Pegasus Airlines",
  "OF": "Transports et Travaux Aériens de Madagascar",
  "FJ": "Fiji Airways",
  "RC": "Atlantic Airways",
  "NY": "Air Iceland",
  "2P": "Air Philippines",
  "2U": "Air Guinee Express",
  "0A": "Amber Air",
  "DA": "Air Georgia",
  "GL": "Miami Air International",
  "LL": "Allegro",
  "5Y": "Atlas Air",
  "GG": "Cargo 360",
  "H9": "Izair",
  "HD": "AIRDO",
  "IP": "Atyrau Air Ways",
  "QK": "Air Canada Jazz",
  "KK": "Atlasjet",
  "JS": "Air Koryo",
  "KC": "Air Astana",
  "LV": "Albanian Airlines",
  "3S": "Air Guyane Express",
  "D4": "Alidaunia",
  "9I": "Thai Sky Airlines",
  "XL": "Aerolane",
  "A6": "Hongtu Airlines",
  "TD": "Tulip Air",
  "L8": "Air Luxor GB",
  "LK": "Air Luxor",
  "MK": "Air Mauritius",
  "MD": "Air Madagascar",
  "9U": "Air Moldova",
  "L9": "Teamline Air",
  "A7": "Air Plus Comet",
  "QO": "Origin Pacific Airways",
  "MR": "Air Mauritanie",
  "F4": "Albarka Air",
  "AJ": "Aero Contractors",
  "8Y": "China Postal Airlines",
  "OT": "Aeropelican Air Services",
  "AD": "Azul Linhas Aéreas Brasileiras",
  "QD": "Air Class Líneas Aéreas",
  "QS": "Travel Service",
  "AG": "",
  "MC": "Air Mobility Command",
  "RE": "Stobart Air",
  "UU": "Air Austral",
  "ZP": "Silk Way Airlines",
  "6K": "Asian Spirit",
  "A5": "Hop!",
  "QL": "Línea Aérea de Servicio Ejecutivo Regional",
  "R3": "Yakutia Airlines",
  "MV": "Armenian International Airways",
  "2O": "Air Salone",
  "U8": "Armavia",
  "BQ": "Baltia Air Lines",
  "P5": "AeroRepública",
  "BF": "Bluebird Cargo",
  "5L": "AeroSur",
  "JR": "Aero California",
  "Z3": "Avient Aviation",
  "GM": "Air Slovakia",
  "VW": "Aeromar",
  "OR": "TUI Airlines Netherlands",
  "CG": "Airlines PNG",
  "TY": "Iberworld",
  "FL": "AirTran Airways",
  "TS": "Air Transat",
  "EC": "Avialeasing Aviation Company",
  "DW": "Aero-Charter Ukraine",
  "U7": "USA Jet Airlines",
  "6R": "Alrosa Air Company",
  "6A": "Aviacsa",
  "JZ": "Skyways Express",
  "AQ": "MAP-Management and Planung",
  "3J": "Zip",
  "SM": "Swedline Express",
  "KJ": "British Mediterranean Airways",
  "BX": "Coast Air",
  "YE": "Aryan Cargo Express",
  "VJ": "Vietjet Air",
  "3O": "Air Arabia Maroc",
  "X9": "Avion Express",
  "JD": "Beijing Capital Airlines",
  "ID": "Interlink Airlines",
  "BA": "British Airways",
  "BG": "Biman Bangladesh Airlines",
  "B4": "Bankair",
  "WX": "CityJet",
  "BZ": "Keystone Air Service",
  "JA": "JetSMART",
  "J4": "Jordan International Air Cargo",
  "8H": "BH Air",
  "4T": "Belair Airlines",
  "UP": "Bahamasair",
  "E6": "Bringer Air Cargo Taxi Aéreo",
  "LZ": "Balkan Bulgarian Airlines",
  "TH": "Transmile Air Services",
  "BS": "British International Helicopters",
  "PG": "Bangkok Airways",
  "KF": "Blue1",
  "JV": "Bearskin Lake Air Service",
  "B3": "Bellview Airlines",
  "BD": "BMI",
  "WW": "WOW air",
  "CH": "Bemidji Airlines",
  "BO": "Bouraq Indonesia Airlines",
  "BV": "Blue Panorama Airlines",
  "7R": "BRA-Transportes Aéreos",
  "8E": "Bering Air",
  "B2": "Belavia Belarusian Airlines",
  "BN": "Horizon Airlines",
  "GQ": "Big Sky Airlines",
  "V9": "Star1 Airlines",
  "Y6": "Batavia Air",
  "BU": "Buryat Airlines Aircompany",
  "J8": "Berjaya Air",
  "QW": "Blue Wings",
  "SN": "Brussels Airlines",
  "DB": "Brit Air",
  "E9": "Boston-Maine Airways",
  "NT": "Binter Canarias",
  "0B": "Blue Air",
  "FB": "Bulgaria Air",
  "8N": "Barents AirLink",
  "CJ": "CityFlyer Express",
  "YB": "Borajet",
  "BW": "Caribbean Airlines",
  "7N": "PAWA Dominicana",
  "5C": "CAL Cargo Air Lines",
  "3C": "RegionsAir",
  "R9": "Camai Air",
  "UY": "Cameroon Airlines",
  "C6": "CanJet",
  "CP": "Compass Airlines",
  "5T": "Canadian North",
  "W2": "Canadian Western Airlines",
  "9K": "Cape Air",
  "PT": "West Air Sweden",
  "2G": "Cargoitalia",
  "W8": "Cargojet Airways",
  "C8": "Chicago Express Airlines",
  "8B": "Caribbean Star Airlines",
  "V3": "Carpatair",
  "CX": "Cathay Pacific",
  "KX": "Cayman Airways",
  "5J": "Cebu Pacific",
  "3B": "Central Connect Airlines",
  "9M": "Central Mountain Air",
  "J7": "ValuJet Airlines",
  "WE": "Thai Smile Airways",
  "OP": "Chalk's International Airlines",
  "MG": "Champion Air",
  "2Z": "Chang An Airlines",
  "S8": "Skywise Airline",
  "CI": "China Airlines",
  "CK": "China Cargo Airlines",
  "MU": "China Eastern Airlines",
  "G5": "China Express Airlines",
  "WH": "WebJet Linhas Aéreas",
  "CZ": "China Southern Airlines",
  "KN": "China United Airlines",
  "XO": "LTE International Airways",
  "3Q": "China Yunnan Airlines",
  "X7": "Chitaavia",
  "QI": "Cimber Sterling",
  "C7": "Rico Linhas Aéreas",
  "C9": "Cirrus Airlines",
  "CF": "City Airline",
  "G3": "Gol Transportes Aéreos",
  "CT": "Civil Air Transport",
  "6P": "Club Air",
  "DQ": "Coastal Air",
  "9L": "Colgan Air",
  "YD": "Cologne Air Transport GmbH",
  "OH": "PSA Airlines",
  "MN": "Comair",
  "C5": "CommutAir",
  "KR": "Comores Airlines",
  "GJ": "Eurofly",
  "DE": "Condor Flugdienst",
  "CO": "Continental Express",
  "CS": "Continental Micronesia",
  "V0": "Conviasa",
  "CM": "Copa Airlines",
  "CQ": "Sunshine Express Airlines",
  "XC": "Corendon Airlines",
  "CD": "Corendon Dutch Airlines",
  "SS": "Corsairfly",
  "XK": "Corse Méditerranée",
  "F5": "Cosmic Air",
  "OU": "Croatia Airlines",
  "7C": "Jeju Air",
  "QE": "Crossair Europe",
  "CU": "Cubana de Aviación",
  "CY": "Cyprus Airways",
  "YK": "Cyprus Turkish Airlines",
  "OK": "Czech Airlines",
  "8L": "Redhill Aviation",
  "XG": "Clickair",
  "WD": "WDL Aviation",
  "DX": "DAT Danish Air Transport",
  "ES": "DHL International",
  "L3": "LTU Austria",
  "D3": "Daallo Airlines",
  "N2": "Kabo Air",
  "H8": "Dalavia",
  "0D": "Darwin Airline",
  "D5": "Dauair",
  "DL": "Delta Air Lines",
  "2A": "Deutsche Bahn",
  "1I": "Sky Trek International Airlines",
  "DH": "Independence Air",
  "Z6": "Dniproavia",
  "YU": "Dominair",
  "DO": "Dominicana de Aviación",
  "E3": "Domodedovo Airlines",
  "D9": "Donavia",
  "KA": "Dragonair, Hong Kong Dragon Airlines",
  "KB": "Druk Air",
  "DI": "Dba",
  "9A": "Eagle Express Air Charter",
  "E1": "Everbread",
  "1C": "Electronic Data Systems",
  "1Y": "Electronic Data Systems",
  "BR": "EVA Air",
  "EY": "Etihad Airways",
  "H7": "Eagle Air",
  "QU": "Uganda Airlines",
  "S9": "East African Safari Air",
  "EA": "European Air Express",
  "T3": "Eastern Airways",
  "QF": "Sunstate Airlines",
  "DK": "Eastland Air",
  "MS": "Egyptair",
  "LY": "El Al Israel Airlines",
  "UZ": "El-Buraq Air Transport",
  "EK": "Emirates Airlines",
  "EM": "Empire Airlines",
  "EU": "Empresa Ecuatoriana De Aviación",
  "E0": "Eos Airlines",
  "B8": "Eritrean Airlines",
  "E7": "European Aviation Air Charter",
  "OV": "SalamAir",
  "ET": "Ethiopian Airlines",
  "RZ": "Euro Exec Express",
  "MM": "SAM Colombia",
  "UI": "Eurocypria Airlines",
  "K2": "Eurolot",
  "3W": "Euromanx Airways",
  "5O": "Europe Airpost",
  "QY": "European Air Transport",
  "EW": "Eurowings",
  "EZ": "Sun Air of Scandinavia",
  "JN": "Excel Airways",
  "MB": "MNG Airlines",
  "OW": "Executive Airlines",
  "EO": "Hewa Bora Airways",
  "U2": "United Feeder Service",
  "DS": "easyJet Switzerland",
  "IH": "Irtysh Air",
  "EF": "Far Eastern Air Transport",
  "FD": "Thai AirAsia",
  "F6": "FaroeJet",
  "F3": "Faso Airways",
  "FX": "Federal Express",
  "N8": "National Air Cargo dba National Airlines",
  "4S": "Finalair Congo",
  "AY": "Finnair",
  "FC": "Finncomm Airlines",
  "FY": "Northwest Regional Airlines",
  "7F": "First Air",
  "DP": "First Choice Airways",
  "8F": "Fischer Air",
  "8D": "Servant Air",
  "B5": "Flightline",
  "PA": "Pan American World Airways",
  "RF": "Florida West International Airways",
  "F2": "Fly Air",
  "OJ": "Overland Airways",
  "SH": "Fly Me Sweden",
  "TE": "Skytaxi",
  "LF": "FlyNordic",
  "F7": "Flybaboo",
  "BE": "Flybe",
  "W3": "Switfair Cargo",
  "VY": "Vueling Airlines",
  "HK": "Four Star Aviation / Four Star Cargo",
  "FH": "Futura International Airways",
  "SJ": "Freedom Air",
  "FP": "Servicios Aéreos de los Andes",
  "F9": "Frontier Airlines",
  "2F": "Frontier Flying Service",
  "FZ": "Flydubai",
  "9Y": "Fly Georgia",
  "VK": "Virgin Nigeria Airways",
  "GX": "Pacificair",
  "Y5": "Pace Airlines",
  "GT": "GB Airways",
  "Z5": "GMG Airlines",
  "7O": "Galaxy Air",
  "1G": "Galileo International",
  "GC": "Gambia International Airlines",
  "G7": "GoJet Airlines",
  "GA": "Garuda Indonesia",
  "4G": "Gazpromavia",
  "A9": "Georgian Airways",
  "ST": "Germania",
  "4U": "Germanwings",
  "GP": "Palau Trans Pacific Airlines",
  "GH": "Ghana Airways",
  "G0": "Ghana International Airlines",
  "GK": "JetStar Japan",
  "DC": "Golden Air",
  "G1": "Gorkha Airlines",
  "ZK": "Great Lakes Airlines",
  "IJ": "Spring Airlines Japan",
  "G6": "Air Volga",
  "J9": "Jazeera Airways",
  "GF": "Gulf Air Bahrain",
  "GY": "Tri-MG Intra Asia Airlines",
  "H6": "Hageland Aviation Services",
  "HR": "Hahn Air",
  "HU": "Hainan Airlines",
  "1R": "Hainan Phoenix Information Systems",
  "2T": "Haiti Ambassador Airlines",
  "4R": "Hamburg International",
  "X3": "Hapag-Lloyd Express (TUIfly)",
  "HF": "Hapagfly",
  "HB": "Harbor Airlines",
  "HQ": "Thomas Cook Airlines",
  "HA": "Hawaiian Airlines",
  "BH": "Hawkair",
  "HN": "Heavylift Cargo Airlines",
  "JB": "Helijet",
  "ZU": "Helios Airways",
  "HW": "North-Wright Airways",
  "2L": "Helvetic Airways",
  "UD": "Hex'Air",
  "5K": "Hi Fly",
  "H5": "Mavial Magadan Airlines",
  "HX": "Hong Kong Airlines",
  "RH": "Republic Express Airlines",
  "UO": "Hong Kong Express Airways",
  "HH": "Hope Air",
  "QX": "Horizon Air",
  "H4": "Inter Island Airways",
  "IK": "Lankair",
  "II": "IBC Airways",
  "0C": "IBL Aviation",
  "C3": "Independent Carrier (ICAR)",
  "1F": "INFINI Travel Information",
  "1U": "Polyot Sirena",
  "IB": "Iberia Airlines",
  "I2": "Iberia Express",
  "FW": "Ibex Airlines",
  "FI": "Icelandair",
  "6E": "IndiGo Airlines",
  "IC": "Indian Airlines",
  "I9": "Indigo Airlines",
  "QZ": "Indonesia AirAsia",
  "IO": "Indonesian Airlines",
  "D6": "Interair South Africa",
  "RS": "Sky Regional Airlines",
  "6I": "International Business Air",
  "3L": "Intersky",
  "I4": "Interstate Airlines",
  "IR": "Iran Air",
  "EP": "Iran Aseman Airlines",
  "IA": "Iraqi Airways",
  "2S": "Satgur Air Transport",
  "CN": "Westward Airways",
  "IF": "Islas Airways",
  "WC": "Islena De Inversiones",
  "6H": "Israir",
  "FS": "Servicios de Transportes Aéreos Fueguinos",
  "GI": "Itek Air",
  "XM": "J-Air",
  "JC": "JAL Express",
  "JO": "Jettime",
  "MT": "Thomas Cook Airlines",
  "1M": "JSC Transport Automated Information Systems",
  "JI": "Midway Airlines (1993–2003)",
  "3X": "Japan Air Commuter",
  "JL": "Japan Airlines Domestic",
  "EG": "Japan Asia Airways",
  "NU": "Japan Transocean Air",
  "O2": "Linear Air",
  "9W": "Jet Airways",
  "QJ": "Jet Airways",
  "PP": "Jet Aviation",
  "3K": "Jetstar Asia Airways",
  "LS": "Jet2.com",
  "B6": "JetBlue Airways",
  "JF": "L.A.B. Flying Service",
  "0J": "Jetclub",
  "SG": "Spicejet",
  "JQ": "Jetstar Airways",
  "JX": "Jett8 Airlines Cargo",
  "R5": "Malta Air Charter",
  "6J": "Skynet Asia Airways",
  "KW": "Wataniya Airways",
  "WA": "Western Airlines",
  "KL": "KLM",
  "K4": "Kalitta Air",
  "K9": "Krylo Airlines",
  "RQ": "Kam Air",
  "E2": "Rio Grande Air",
  "V2": "Vision Airlines",
  "KV": "Kavminvodyavia",
  "M5": "Kenmore Air",
  "KQ": "Kenya Airways",
  "IT": "Tigerair Taiwan",
  "Y9": "Kish Air",
  "KP": "Kiwi International Air Lines",
  "7K": "Kogalymavia Air Company",
  "8J": "Komiinteravia",
  "KE": "Korean Air",
  "7B": "Krasnojarsky Airlines",
  "GW": "SkyGreece Airlines",
  "KU": "Kuwait Airways",
  "GO": "Kuzu Airlines Cargo",
  "N5": "Skagway Air Service",
  "R8": "Kyrgyzstan Airlines",
  "YQ": "TAR Aerolineas",
  "LR": "LACSA",
  "KG": "LAI - Línea Aérea IAACA",
  "LA": "LATAM Chile",
  "4M": "LATAM Argentina",
  "PZ": "TAM Mercosur",
  "LU": "LATAM Express",
  "LP": "LATAM Peru",
  "NI": "Portugalia",
  "L5": "Lufttransport",
  "LO": "LOT Polish Airlines",
  "LT": "LongJiang Airlines",
  "N6": "Nuevo Continente",
  "QV": "Lao Airlines",
  "L7": "Línea Aérea SAPSA",
  "NG": "Lauda Air",
  "LQ": "Lebanese Air Transport",
  "LI": "Leeward Islands Air Transport",
  "LN": "Libyan Arab Airlines",
  "TM": "Linhas Aéreas de Moçambique",
  "JT": "Lion Mentari Airlines",
  "LM": "Livingston",
  "LB": "Lloyd Aéreo Boliviano",
  "LC": "Varig Logística",
  "HE": "Luftfahrtgesellschaft Walter",
  "LH": "Lufthansa Cargo",
  "CL": "Lufthansa CityLine",
  "L1": "Lufthansa Systems",
  "DV": "Nantucket Airlines",
  "LG": "Luxair",
  "5V": "Lviv Airlines",
  "L2": "Lynden Air Cargo",
  "L4": "Lynx Aviation",
  "Z8": "Línea Aérea Amaszonas",
  "MJ": "Mihin Lanka",
  "Q2": "Maldivian (airline)",
  "OD": "Malindo Airways",
  "M7": "Tropical Airways",
  "MH": "Malaysia Airlines",
  "IN": "NAM Air",
  "OM": "MIAT Mongolian Airlines",
  "DM": "Maersk",
  "W5": "Mahan Air",
  "M2": "MHS Aviation GmbH",
  "TF": "Malmö Aviation",
  "MA": "Malév Hungarian Airlines",
  "RI": "Mandala Airlines",
  "JE": "Mango",
  "MP": "Martinair",
  "Q4": "Starlink Aviation",
  "8M": "Myanmar Airways International",
  "MY": "Midwest Airlines (Egypt)",
  "MW": "Mokulele Airlines",
  "7M": "Mayair",
  "M8": "Mekong Airlines",
  "IM": "Menajet",
  "IG": "Meridiana",
  "MZ": "Merpati Nusantara Airlines",
  "YV": "Mesa Airlines",
  "XJ": "Thai AirAsia X",
  "MX": "Mexicana de Aviación",
  "ME": "Middle East Airlines",
  "YX": "Midwest Airlines",
  "M4": "Mistral Air",
  "2M": "Moldavian Airlines",
  "8I": "Myway Airlines",
  "YM": "Montenegro Airlines",
  "5M": "Sibaviatrans",
  "3R": "Moskovia Airlines",
  "M9": "Motor Sich",
  "N4": "Nordwind Airlines",
  "VZ": "Thai Vietjet Air",
  "UB": "Myanma Airways",
  "6N": "Nordic Regional",
  "P9": "Peruvian Airlines",
  "UE": "Transeuropean Airlines",
  "N7": "National Airlines",
  "NA": "North American Airlines",
  "9O": "National Airways Cameroon",
  "NC": "Northern Air Cargo",
  "CE": "Nationwide Airlines",
  "1N": "Navitaire",
  "RA": "Nepal Airlines",
  "EJ": "New England Airlines",
  "2N": "Yuzhmashavia",
  "HG": "Niki",
  "KZ": "Nippon Cargo Airlines",
  "DD": "Nok Air",
  "5N": "Nordavia",
  "JH": "Nordeste Linhas Aéreas Regionais",
  "N9": "North Coast Aviation",
  "NW": "Northwest Airlines",
  "J3": "Northwestern Air",
  "DY": "Norwegian Air Shuttle",
  "D8": "Norwegian Air International",
  "DU": "Norwegian Long Haul",
  "BJ": "Nouvel Air Tunisie",
  "O9": "Nova Airline",
  "VQ": "Novo Air",
  "NK": "Spirit Airlines",
  "UQ": "O'Connor Airlines",
  "O8": "Oasis Hong Kong Airlines",
  "VC": "Voyageur Airways",
  "OA": "Olympic Airlines",
  "WY": "Oman Air",
  "8Q": "Princess Air",
  "R2": "Orenburg Airlines",
  "OX": "Orient Thai Airlines",
  "OL": "OLT Express Germany",
  "ON": "Our Airline",
  "O7": "Ozjet Airlines",
  "PV": "St Barth Commuter",
  "9Q": "PB Air",
  "PU": "PLUNA",
  "U4": "PMTair",
  "BL": "Pacific Airlines",
  "8P": "Pacific Coastal Airlines",
  "Q8": "Pacific East Asia Cargo Airlines",
  "LW": "Sun Air International",
  "PK": "Pakistan International Airlines",
  "PF": "Palestinian Airlines",
  "NR": "Pamir Airways",
  "PN": "Pan American Airways",
  "I7": "Paramount Airways",
  "P3": "Passaredo Transportes Aéreos",
  "KS": "Peninsula Airways",
  "Z2": "EZD",
  "PR": "Philippine Airlines",
  "9R": "Phuket Air",
  "PI": "Sun Air (Fiji)",
  "9E": "Pinnacle Airlines",
  "PO": "Polar Air Cargo",
  "PH": "Polynesian Airlines",
  "PD": "Porter Airlines",
  "BK": "Potomac Air",
  "PW": "Precision Air",
  "TO": "Transavia France",
  "FE": "Primaris Airlines",
  "8W": "Private Wings Flugcharter",
  "P6": "Privilege Style Líneas Aéreas",
  "P0": "Proflight Zambia",
  "FV": "Rossiya",
  "QR": "Qatar Airways",
  "R6": "RACSA",
  "1D": "Radixx Solutions International",
  "V4": "Vieques Air Link",
  "FN": "Regional Air Lines",
  "ZL": "Regional Express",
  "P7": "Russian Sky Airlines",
  "RW": "Republic Airlines",
  "RL": "Royal Phnom Penh Airways",
  "SL": "Thai Lion Mentari",
  "GZ": "Air Rarotonga",
  "RR": "Royal Air Force",
  "AT": "Royal Air Maroc",
  "R0": "Royal Airlines",
  "BI": "Royal Brunei Airlines",
  "RJ": "Royal Jordanian",
  "RK": "Royal Khmer Airlines",
  "WR": "WestJet Encore",
  "WB": "Rwandair Express",
  "7S": "Ryan Air Service",
  "RD": "Ryan International Airlines",
  "FR": "Ryanair",
  "YS": "Régional Compagnie Aérienne Européenne",
  "TR": "Tigerair Singapore",
  "6Y": "Smartlynx Airlines",
  "7E": "Sylt Air GmbH",
  "S4": "SATA International",
  "SA": "South African Airways",
  "W7": "Western Pacific Airlines",
  "NL": "Shaheen Air International",
  "SK": "SAS Braathens",
  "UG": "Tuninter",
  "S7": "S7 Airlines",
  "BB": "Seaborne Airlines",
  "K5": "SeaPort Airlines",
  "UL": "SriLankan Airlines",
  "SY": "Sun Country Airlines",
  "I6": "Sky Eyes",
  "7G": "Star Flyer",
  "FA": "Safair",
  "HZ": "Sat Airlines",
  "SP": "SATA Air Acores",
  "8S": "Scorpio Aviation",
  "ZY": "Sky Airlines",
  "SQ": "Singapore Airlines Cargo",
  "SI": "Skynet Airlines",
  "XS": "SITA",
  "FT": "Siem Reap Airways",
  "SX": "Skybus Airlines",
  "S6": "Star Air",
  "D2": "Severstal Air Company",
  "5G": "Skyservice Airlines",
  "SD": "Sudan Airways",
  "SV": "Saudia",
  "WN": "Southwest Airlines",
  "A4": "Southern Winds Airlines",
  "WG": "Sunwing Airlines",
  "LX": "Swiss International Air Lines",
  "SR": "Swissair",
  "WV": "Swe Fly",
  "XQ": "SunExpress",
  "AL": "TransAVIAexport Airlines",
  "E5": "Samara Airlines",
  "SC": "Shandong Airlines",
  "9C": "Spring Airlines",
  "3U": "Sichuan Airlines",
  "FM": "Shanghai Airlines",
  "ZH": "Shenzhen Airlines",
  "7L": "Sun D'Or",
  "NE": "SkyEurope",
  "SO": "Sunshine Airlines",
  "JK": "Spanair",
  "1Z": "Sabre Pacific",
  "1S": "Sabre",
  "1H": "Siren-Travel",
  "1Q": "Sirena",
  "1K": "Sutra",
  "2C": "SNCF",
  "S0": "Slok Air Gambia",
  "S3": "Santa Barbara Airlines",
  "H2": "Sky Airline",
  "OO": "SkyWest Airlines",
  "BC": "Skymark Airlines",
  "LJ": "Sierra National Airlines",
  "MI": "SilkAir",
  "6Q": "Slovak Airlines",
  "PY": "Surinam Airways",
  "NB": "Sterling Airlines",
  "IE": "Solomon Airlines",
  "6W": "Saratov Airlines Joint Stock Company",
  "S5": "Trast Aero",
  "R1": "Sirin",
  "O3": "SF Airlines",
  "EQ": "TAME",
  "TP": "TAP Portugal",
  "TU": "Tunisair",
  "3V": "TNT Airways",
  "T2": "Thai Air Cargo",
  "TQ": "Tandem Aero",
  "ZT": "Titan Airways",
  "DG": "Tigerair Philippines",
  "TG": "Thai Airways International",
  "TK": "Turkish Airlines",
  "T7": "Twin Jet",
  "3P": "Tiara Air",
  "TI": "Tol-Air Services",
  "BY": "Thomson Airways",
  "PM": "Tropic Air",
  "QT": "TAMPA",
  "K3": "Taquan Air Services",
  "GE": "TransAsia Airways",
  "HV": "Transavia Holland",
  "VR": "TACV",
  "T9": "Thai Star Airlines",
  "9T": "Transwest Air",
  "UN": "Transaero Airlines",
  "T5": "Turkmenhovayollary",
  "T6": "Tavrey Airlines",
  "TW": "T'way Air",
  "6B": "TUIfly Nordic",
  "DT": "TAAG Angola Airlines",
  "SF": "Tassili Airlines",
  "TJ": "Tradewind Aviation",
  "1E": "Travelsky Technology",
  "2H": "Thalys",
  "1L": "Open Skies Consultative Commission",
  "RO": "Tarom",
  "3T": "Turan Air",
  "T4": "TRIP Linhas Aéreas",
  "L6": "Tbilaviamsheni",
  "XN": "XpressAir",
  "VO": "Tyrolean Airways",
  "U5": "USA3000 Airlines",
  "UA": "United Airlines",
  "4H": "United Airways",
  "U6": "Ural Airlines",
  "UF": "UM Airlines",
  "6Z": "Ukrainian Cargo Airways",
  "5X": "United Parcel Service",
  "US": "US Airways",
  "UH": "US Helicopter",
  "UT": "UTair Aviation",
  "HY": "Uzbekistan Airways",
  "PS": "Ukraine International Airlines",
  "VA": "Viasa",
  "VF": "Valuair",
  "0V": "Vietnam Air Services Company (VASCO)",
  "VN": "Vietnam Airlines",
  "NN": "VIM Airlines",
  "2R": "Via Rail Canada",
  "Y4": "Volaris",
  "VI": "Volga-Dnepr Airlines",
  "TV": "Virgin Express",
  "VS": "Virgin Atlantic Airways",
  "ZG": "Viva Macau",
  "XF": "Vladivostok Air",
  "VM": "Viaggio Air",
  "9V": "Vipair Airlines",
  "RG": "VRG Linhas Aéreas",
  "VP": "VASP",
  "VG": "VLM Airlines",
  "WT": "Wasaya Airways",
  "2W": "Welcome Air",
  "WZ": "West African Airlines",
  "YH": "West Caribbean Airways",
  "8O": "West Coast Air",
  "WS": "WestJet",
  "XP": "Xtra Airways",
  "WF": "Widerøe",
  "IV": "Wind Jet",
  "7W": "Windrose Air",
  "8Z": "Wizz Air Bulgaria",
  "W6": "Wizz Air",
  "WO": "World Airways",
  "1P": "Worldspan",
  "MF": "Xiamen Airlines",
  "SE": "XL Airways France",
  "YL": "Yamal Airlines",
  "Y8": "Yangtze River Express",
  "Y0": "Yellow Air Taxi/Friendship Airways",
  "IY": "Yemenia",
  "C4": "Zimex Aviation",
  "Q3": "Zambian Airways",
  "Z4": "Zoom Airlines"
}

In [82]:
df["Reporting_Airline_Name"] = df["Reporting_Airline"].replace(airline_code_dictory)

In [84]:
df.loc[~df["Is_LCC"], "Reporting_Airline_Name"].unique()

array(['Northwest Airlines', 'American Eagle Airlines', 'Delta Air Lines',
       'Pinnacle Airlines', 'American Airlines', 'SkyWest Airlines',
       'United Airlines', 'Sun Air (Fiji)', 'Alaska Airlines, Inc.',
       'ExpressJet', 'PSA Airlines', 'Phoenix Airways',
       'Hawaiian Airlines', 'XE', 'European Air Express',
       'Aloha Air Cargo', 'PA (1)', 'ML (1)'], dtype=object)

 - Northwest Airlines - Likely Full Service Carrier
     - https://transportation.northwestern.edu/docs/research/core-topics/transportation-demand-economics-and-forecasting/Gordon_HubandNetwork.pdf 
     - " Northwest has very few point-to-point domestic routes which do not pass through one or two of the three
hubs."
 - American Airlines -  Full Service Carrier (https://simpleflying.com/)
 - American Eagle Airlines  -  Full Service Carrier (https://simpleflying.com/)
 - Delta Air Lines - Full Service Carrier (https://simpleflying.com/)
 - Pinnacle Airlines - UNKNWON
    - https://www.sec.gov/Archives/edgar/data/1166291/000095014407001984/g05885e10vk.htm
    - Provided service to northwest hubs?
 - SkyWest Airlines - UNKNOWN 
    - https://en.m.wikipedia.org/wiki/SkyWest_Airlines#:~:text=SkyWest%20Airlines%20is%20an%20American,by%20four%20partner%20mainline%20airlines.
    - Regional carrier????
 - United Airlines - Full Service Carrier (https://simpleflying.com/)
 - Sun Air (Fiji)- Full Service Carrier (https://simpleflying.com/)
 - Alaska Airlines, Inc. - Full Service Carrier (https://simpleflying.com/)
 - ExpressJet - UNKNOWN
     - operated point to point
     https://en.wikipedia.org/wiki/ExpressJet

 - PSA Airlines - UNKNWON
 - Phoenix Airways - UNKNWON
 - Hawaiian Airlines - UNKNOWN
 - XE - UNKNOWN
 - European Air Express - UNKNWON
 - Aloha Air Cargo - UNKNWON
 - PA 
 - ML 



 New approch find a list of point to point buinsesses, and whell and spoke hub carreirs
 

### New idea: estimate if the airport is a point to point or hub based model with our dataset!

Lets look at the origin and arrival of each place, get thier value counts, and estimate how many hubs a given airline has

Reserach has shown networks are complicated and can't be as easily grouped so instead lets find some proxy to study this network complicity idea

A network that is more point to point will have fewer major airports

A network that is more hub and spoke will have one major airports, and are fewer alterantive fights

In [139]:
df = airline_data.loc[~airline_data["DepDelay"].isnull(), ["DepDelay", "Reporting_Airline", "Origin", "Dest"]]
df["Reporting_Airline_Name"] = df["Reporting_Airline"].replace(airline_code_dictory)

In [143]:
def network_study(df):
    Airports = df["Origin"].value_counts() + df["Dest"].value_counts()
    Airports = Airports.reset_index().T

    Airports.columns = Airports.loc["index"]
    return Airports.drop("index")

airport_dist = df.groupby("Reporting_Airline_Name").apply(network_study).reset_index().set_index("Reporting_Airline_Name").drop("level_1", axis=1)

In [144]:
airport_dist

index,BOS,CLT,DCA,DEN,DFW,EWR,FLL,HNL,HOU,IND,...,VCT,VEL,VIS,WYS,XWA,YKM,DET,LBE,UCA,CCR
Reporting_Airline_Name,,,,,,,,,,,,,,,,,,,,,
ATA Airlines,86.0,62.0,88.0,95.0,158.0,98.0,52.0,134.0,12.0,297.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AirTran Airways,1579,411,684,304,633,296,1433,NaN,473,898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Alaska Airlines, Inc.",443.0,NaN,471.0,698.0,316.0,401.0,83.0,559.0,NaN,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Allegiant Air,NaN,NaN,NaN,6.0,NaN,22.0,128.0,NaN,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aloha Air Cargo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1298,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
American Airlines,9637.0,10799.0,8258.0,4568.0,100909.0,5434.0,3198.0,1848.0,887.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
American Eagle Airlines,3475.0,1211.0,2339.0,189.0,26888.0,606.0,21.0,NaN,763.0,811.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Continental Express,3982.0,235.0,3049.0,9563.0,2206.0,29000.0,2427.0,782.0,454.0,1043.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Delta Air Lines,11335.0,2388.0,7484.0,4567.0,22310.0,4245.0,7372.0,1776.0,1455.0,2433.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:


airport_dist = airport_dist.apply(lambda x: x/x.sum(), axis=1)

In [165]:
((airport_dist.quantile(q=0.9, axis=1) - airport_dist.quantile(q=0.1, axis=1))/airport_dist.sum(axis=1)).sort_values(ascending=False)

Reporting_Airline_Name
Aloha Air Cargo                   0.182848
Hawaiian Airlines                 0.106041
PA (1)                            0.077850
Ukraine International Airlines    0.071179
Virgin America                    0.060843
Spirit Airlines                   0.052729
ATA Airlines                      0.036568
ML (1)                            0.032667
Midwest Airlines                  0.031388
Alaska Airlines, Inc.             0.026026
Southwest Airlines                0.025940
AirTran Airways                   0.023379
JetBlue Airways                   0.023312
Sun Air (Fiji)                    0.022947
European Air Express              0.021929
Phoenix Airways                   0.016794
American Airlines                 0.015357
Frontier Airlines                 0.014433
Continental Express               0.013477
Allegiant Air                     0.012976
T'way Air                         0.011927
US Airways                        0.011485
Independence Air               

In [164]:
airport_dist.max(axis=1).sort_values(ascending=False)

Reporting_Airline_Name
ML (1)                            0.473333
Hawaiian Airlines                 0.447536
Aloha Air Cargo                   0.420065
T'way Air                         0.404272
Frontier Airlines                 0.367140
ATA Airlines                      0.359832
AirTran Airways                   0.323044
Phoenix Airways                   0.320252
European Air Express              0.310708
Virgin America                    0.302954
Independence Air                  0.285138
Alaska Airlines, Inc.             0.283763
Delta Air Lines                   0.246978
Continental Express               0.231581
XE                                0.227840
ExpressJet                        0.227092
Ukraine International Airlines    0.225204
American Airlines                 0.219012
PSA Airlines                      0.216481
Northwest Airlines                0.212699
JetBlue Airways                   0.202845
Sun Air (Fiji)                    0.196474
United Airlines                

In [161]:
(airport_dist > 0.05).sum(axis=1).sort_values(ascending=False)

Reporting_Airline_Name
Spirit Airlines                   6
Ukraine International Airlines    6
PA (1)                            6
Allegiant Air                     5
Aloha Air Cargo                   5
SkyWest Airlines                  5
Hawaiian Airlines                 5
Mesa Airlines                     4
Pinnacle Airlines                 4
JetBlue Airways                   4
Southwest Airlines                4
Midwest Airlines                  4
Continental Express               4
United Airlines                   4
Virgin America                    4
Alaska Airlines, Inc.             4
Sun Air (Fiji)                    3
US Airways                        3
ATA Airlines                      3
Northwest Airlines                3
AirTran Airways                   3
Independence Air                  3
European Air Express              3
American Airlines                 3
XE                                3
PSA Airlines                      2
Phoenix Airways                   2
Expre

In [169]:
max_node_cap = airport_dist.max(axis=1).sort_values(ascending=False)
max_node_cap > max_node_cap.median()

Reporting_Airline_Name
ML (1)                             True
Hawaiian Airlines                  True
Aloha Air Cargo                    True
T'way Air                          True
Frontier Airlines                  True
ATA Airlines                       True
AirTran Airways                    True
Phoenix Airways                    True
European Air Express               True
Virgin America                     True
Independence Air                   True
Alaska Airlines, Inc.              True
Delta Air Lines                    True
Continental Express                True
XE                                 True
ExpressJet                         True
Ukraine International Airlines    False
American Airlines                 False
PSA Airlines                      False
Northwest Airlines                False
JetBlue Airways                   False
Sun Air (Fiji)                    False
United Airlines                   False
American Eagle Airlines           False
Pinnacle Airlines

In [171]:
df["prop_of_carrier_flights_in_carrier_hub"] = df["Reporting_Airline_Name"].apply(lambda x: (max_node_cap > max_node_cap.median()).loc[x])

In [172]:
df

,DepDelay,Reporting_Airline,Origin,Dest,Reporting_Airline_Name,prop_of_carrier_flights_in_carrier_hub
0,19.0,NW,MSP,SLC,Northwest Airlines,False
1,-2.0,FL,MKE,MCO,AirTran Airways,True
2,14.0,MQ,GJT,DFW,American Eagle Airlines,False
3,0.0,DL,LAX,DTW,Delta Air Lines,True
4,51.0,US,EWR,CLT,US Airways,False
...,...,...,...,...,...,...
1999995,4.0,WN,LAS,PHX,Southwest Airlines,False
1999996,0.0,CO,EWR,DFW,Continental Express,True
1999997,-6.0,US,CHS,CLT,US Airways,False
1999998,8.0,WN,MDW,BNA,Southwest Airlines,False


In [174]:
import scipy.stats as stats
stats.ttest_ind(
    df.loc[df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"], 
    df.loc[~df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"], 
    equal_var=False,
    alternative='less'
),

(Ttest_indResult(statistic=-15.1470320300718, pvalue=4.0047549378976405e-52),)

In [176]:
df.loc[~df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"].mean() - df.loc[df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"].mean()

0.7562515131995244

In [178]:
delay_df = df[df["DepDelay"] > 0]

import scipy.stats as stats
stats.ttest_ind(
    delay_df.loc[delay_df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"], 
    delay_df.loc[~delay_df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"], 
    equal_var=False,
    alternative='less'
),

(Ttest_indResult(statistic=-6.740507678613449, pvalue=7.900722470642686e-12),)

In [189]:
delay_df.loc[~delay_df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"].mean() - delay_df.loc[delay_df["prop_of_carrier_flights_in_carrier_hub"], "DepDelay"].mean()


0.7740277945322305

26.051162623626045